In [ ]:
# Importar bibliotecas

import os
import re
import io
import sys
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

from dotenv import load_dotenv, find_dotenv
from pathlib import Path

In [2]:
# FUNÇÕES DE PREPARO

def coerce_numeric(s):
    return pd.to_numeric(s, errors="coerce")

def extrair_fase(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if "alfa" in valor:
        return 0
    m = re.search(r"fase\s*(\d+)", valor)
    if m:
        return int(m.group(1))
    return np.nan

def padronizar_genero(df):
    df = df.copy()
    if "genero" in df.columns:
        df["genero"] = df["genero"].astype(str).str.strip().str.lower()
        map_genero = {
            "menino": "masculino",
            "masculino": "masculino",
            "menina": "feminino",
            "feminino": "feminino"
        }
        df["genero"] = df["genero"].map(map_genero)
    return df

def padronizar_idade(df):
    df = df.copy()
    if "idade" not in df.columns:
        return df

    s = df["idade"]
    dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        dt.notna() & (dt.dt.year == 1900) & (dt.dt.month == 1),
        dt.dt.day,
        np.nan
    )

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_final = pd.Series(idade_num, index=df.index)

    mask = idade_final.isna() & ~pd.isna(idade_from_date)
    idade_final.loc[mask] = idade_from_date[mask]

    # faixa plausível
    idade_final = idade_final.where(idade_final.between(6, 30))
    df["idade"] = idade_final.round()
    return df

def tratar_inde_2024(df):
    df = df.copy()
    if "inde_2024" in df.columns:
        tmp = df["inde_2024"].astype(str).str.strip().str.upper()
        tmp = tmp.replace("INCLUIR", np.nan)
        df["inde_2024"] = coerce_numeric(tmp)
    return df

def preparar_base(df, modo_treino: bool):
    """
    modo_treino=True:
      - cria target a partir de 'ian'
      - remove vazamento (ian/defasagem)
    modo_treino=False:
      - não cria target
      - remove vazamento se existir
    """
    df = df.copy()

    df = padronizar_genero(df)
    df = padronizar_idade(df)
    df = tratar_inde_2024(df)

    if "fase_ideal" in df.columns:
        df["fase_ideal"] = df["fase_ideal"].apply(extrair_fase)

    # Target (somente treino)
    if modo_treino:
        if "ian" not in df.columns:
            raise ValueError("modo_treino=True exige coluna 'ian' para criar a target.")
        df["risco_defasagem_atual"] = (pd.to_numeric(df["ian"], errors="coerce") <= 5).astype(int)

    # Remover vazamento sempre que existir
    df = df.drop(columns=[c for c in ["ian", "defasagem"] if c in df.columns], errors="ignore")

    # Remoções iguais ao treino (mantém seu padrão)
    colunas_para_remover = ["ra",
                            "nome",
                            "data_nasc",
                            "escola",
                            "avaliador_1",
                            "avaliador_2",
                            "avaliador_3",
                            "avaliador_4",
                            "avaliador_5",
                            "avaliador_6",
                            "rec_av1",
                            "rec_av2",
                            "rec_av3",
                            "rec_av4",
                            "rec_av5",
                            "rec_av6",
                            "rec_psicologia",
                            "indicado",
                            "atingiu_pv",
                            "destaque_ieg",
                            "destaque_ida",
                            "destaque_ivp",
                            "pedra_2020",
                            "pedra_2021",
                            "pedra_2022",
                            "pedra_2023",
                            "pedra_2024",
                            "fase",
                            "turma",
                            "instituicao_ensino",
                            "ativo_inativo",
                            "cg",
                            "cf",
                            "ct",
                            "inde_2024",
                            "ano_ingresso"
    ]
    df = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors="ignore")

    # Features extras
    cols_acad = [c for c in ["mat","por","ing"] if c in df.columns]
    if len(cols_acad) >= 2:
        df["media_academica"] = df[cols_acad].mean(axis=1)

    cols_comp = [c for c in ["iaa","ieg","ips","ipp"] if c in df.columns]
    if len(cols_comp) >= 2:
        df["media_comportamental"] = df[cols_comp].mean(axis=1)

    if ("inde_2022" in df.columns) and ("inde_2023" in df.columns):
        df["delta_inde"] = df["inde_2023"] - df["inde_2022"]

    return df

def make_preprocess(X_train: pd.DataFrame) -> ColumnTransformer:
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]

    numeric_pipe = Pipeline(steps=[
        ("imputer", KNNImputer(n_neighbors=7, weights="distance")),
        ("scaler", StandardScaler())
    ])

    categorical_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", categorical_pipe, cat_cols),
        ],
        remainder="drop"
    )
    return preprocess

In [ ]:
# TREINO COMPLETO + TESTE + THRESHOLD + SALVAR ARQUIVOS

def treinar_e_salvar(excel_path: str,
                     out_model_path: str = "modelo_passos_magicos.pkl",
                     out_cfg_path: str = "config_passos_magicos.pkl",
                     seed: int = 42):

    base = pd.read_excel(excel_path)
    print("Shape original:", base.shape)

    base2 = preparar_base(base, modo_treino=True)
    print("Shape após preparo:", base2.shape)
    print("Target balance:\n", base2["risco_defasagem_atual"].value_counts())

    if "ano_pede" not in base2.columns:
        raise ValueError("Coluna 'ano_pede' não encontrada após preparo. Precisa dela para split temporal.")

    train_df = base2[base2["ano_pede"] < 2024].copy()
    test_df  = base2[base2["ano_pede"] == 2024].copy()

    # Remover 'risco_defasagem_atual' e 'ano_pede' para que o algoritmo não aprenda o viés temporal
    colunas_para_ignorar = ["risco_defasagem_atual", "ano_pede"]

    X_train = train_df.drop(columns=colunas_para_ignorar)
    y_train = train_df["risco_defasagem_atual"].astype(int)

    X_test = test_df.drop(columns=colunas_para_ignorar)
    y_test = test_df["risco_defasagem_atual"].astype(int)

    print("Treino:", X_train.shape, "Teste:", X_test.shape)

    preprocess = make_preprocess(X_train)

    models = {
        "LogReg": LogisticRegression(max_iter=2000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(
            n_estimators=400, random_state=seed,
            class_weight="balanced_subsample"
        ),
        "MLP": MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            max_iter=500,
            random_state=seed
        ),
    }

    if HAS_XGB:
        models["XGBoost"] = XGBClassifier(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=seed,
            eval_metric="logloss"
        )
    else:
        print("XGBoost não disponível. Seguindo sem XGBoost.")

    # CV no treino
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scoring = {"acc": "accuracy", "roc_auc": "roc_auc", "pr_auc": "average_precision"}

    print("\n================= CV (TREINO) =================")
    cv_results = []
    pipes = {}

    for name, clf in models.items():
        pipe = Pipeline(steps=[("prep", preprocess), ("model", clf)])
        pipes[name] = pipe

        scores = cross_validate(
            pipe, X_train, y_train,
            cv=cv, scoring=scoring,
            n_jobs=-1, return_train_score=False
        )

        row = {
            "model": name,
            "acc_mean": scores["test_acc"].mean(),
            "acc_std": scores["test_acc"].std(),
            "roc_auc_mean": scores["test_roc_auc"].mean(),
            "roc_auc_std": scores["test_roc_auc"].std(),
            "pr_auc_mean": scores["test_pr_auc"].mean(),
            "pr_auc_std": scores["test_pr_auc"].std(),
        }
        cv_results.append(row)

        print(f"\n{name}")
        print(f"  ACC     : {row['acc_mean']:.4f} ± {row['acc_std']:.4f}")
        print(f"  ROC AUC : {row['roc_auc_mean']:.4f} ± {row['roc_auc_std']:.4f}")
        print(f"  PR AUC  : {row['pr_auc_mean']:.4f} ± {row['pr_auc_std']:.4f}")

    cv_df = pd.DataFrame(cv_results).sort_values("roc_auc_mean", ascending=False)
    print("\nResumo CV:")
    print(cv_df)

    # Teste final 2024
    print("\n================= TESTE FINAL (2024) =================")
    test_rows = []

    def avaliar(nome, pipe):
        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = pipe.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba)
        pr  = average_precision_score(y_test, y_proba)

        print(f"\n>>> {nome}")
        print(f"ACC     : {acc:.4f}")
        print(f"ROC AUC : {roc:.4f}")
        print(f"PR AUC  : {pr:.4f}")
        print("\nMatriz de confusão:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification report:")
        print(classification_report(y_test, y_pred, digits=4))

        return {"model": nome, "acc": acc, "roc_auc": roc, "pr_auc": pr}

    for name, pipe in pipes.items():
        test_rows.append(avaliar(name, pipe))

    test_df_res = pd.DataFrame(test_rows).sort_values("roc_auc", ascending=False)
    print("\nResumo TESTE (ordenado por ROC AUC):")
    print(test_df_res)

    best_name = test_df_res.iloc[0]["model"]
    best_pipe = pipes[best_name]
    print("\nMelhor modelo no TESTE:", best_name)

    # Ajuste threshold no MELHOR modelo do teste
    best_pipe.fit(X_train, y_train)
    y_proba_best = best_pipe.predict_proba(X_test)[:, 1]

    thresholds = np.linspace(0.10, 0.90, 17)
    thr_rows = []
    for t in thresholds:
        y_pred_t = (y_proba_best >= t).astype(int)
        report = classification_report(y_test, y_pred_t, output_dict=True, zero_division=0)
        acc = accuracy_score(y_test, y_pred_t)
        prec1 = report["1"]["precision"]
        rec1 = report["1"]["recall"]
        f11 = report["1"]["f1-score"]
        thr_rows.append([t, acc, prec1, rec1, f11])

    thr_df = pd.DataFrame(thr_rows, columns=["threshold","accuracy","precision_risco","recall_risco","f1_risco"])
    print("\nTabela de thresholds (classe RISCO=1):")
    print(thr_df)

    best_thr_row = thr_df.sort_values("f1_risco", ascending=False).iloc[0]
    best_threshold = float(best_thr_row["threshold"])

    print("\nMelhor threshold (por F1 da classe 1):", best_threshold)
    print("Linha escolhida:\n", best_thr_row)

    # Salvar arquivos
    joblib.dump(best_pipe, out_model_path)
    joblib.dump({"threshold": best_threshold, "best_model": best_name}, out_cfg_path)

    print("\nArquivos salvos:")
    print(f"- (pipeline completo)")
    print(f"- (threshold + nome do modelo)")

    return best_name, best_threshold, cv_df, test_df_res, thr_df

In [ ]:
# GERER TREINAMENTO

# Definir caminho da variável de ambiente
load_dotenv(find_dotenv())
data_path = Path(os.getenv("DATA_PATH"))
models_dir = Path(os.getenv("MODELS"))

MODO = "treino"  # "treino" ou "app"
EXCEL_PATH = data_path 
MODEL_PATH = models_dir / "modelo_passos_magicos.pkl"
CFG_PATH = models_dir / "config_passos_magicos.pkl"

if MODO == "treino":
    treinar_e_salvar(EXCEL_PATH, MODEL_PATH, CFG_PATH)

elif MODO == "app":
    print("Modo app: rode em ambiente local com Streamlit.")
    print("Exemplo: streamlit run projeto_completo.py")
    
else:
    raise ValueError("MODO inválido. Use 'treino' ou 'app'.")

Shape original: (3030, 54)
Shape após preparo: (3030, 20)
Target balance:
 risco_defasagem_atual
1    1687
0    1343
Name: count, dtype: int64
Treino: (1874, 18) Teste: (1156, 18)

================= CV (TREINO) =================

LogReg
  ACC     : 0.9093 ± 0.0106
  ROC AUC : 0.9429 ± 0.0122
  PR AUC  : 0.9256 ± 0.0209

RandomForest
  ACC     : 0.9125 ± 0.0040
  ROC AUC : 0.9675 ± 0.0038
  PR AUC  : 0.9764 ± 0.0036

MLP
  ACC     : 0.9637 ± 0.0088
  ROC AUC : 0.9915 ± 0.0030
  PR AUC  : 0.9940 ± 0.0024

XGBoost
  ACC     : 0.9365 ± 0.0072
  ROC AUC : 0.9846 ± 0.0035
  PR AUC  : 0.9895 ± 0.0026

Resumo CV:
          model  acc_mean   acc_std  roc_auc_mean  roc_auc_std  pr_auc_mean  \
2           MLP  0.963705  0.008754      0.991487     0.003007     0.994035   
3       XGBoost  0.936501  0.007225      0.984575     0.003456     0.989474   
1  RandomForest  0.912483  0.003994      0.967544     0.003776     0.976442   
0        LogReg  0.909288  0.010644      0.942913     0.012153     0.92

In [ ]:
# NOMES DAS ETAPAS QUE COMPOEM O PIPELINE

list(pipe.named_steps.keys())

['prep', 'model']

In [ ]:
# GERAR FEATURE IMPORTANTES

model = pipe.named_steps['model']

if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
elif hasattr(model, 'coef_'):
    importances = np.abs(model.coef_[0])
else:
    importances = None
    print("Este modelo não suporta extração de importância de variáveis.")

if importances is not None:
    features = pipe.named_steps['prep'].get_feature_names_out()
    imp = pd.DataFrame({
        'feature': features,
        'importance': importances
    }).sort_values('importance', ascending=False)

    print(imp.head(15))

                      feature  importance
0                  num__idade    0.135120
13            num__fase_ideal    0.112063
1              num__inde_2022    0.111861
2              num__inde_2023    0.088159
3                   num__n_av    0.080905
15  num__media_comportamental    0.053673
14       num__media_academica    0.044108
7                    num__ipp    0.043956
11                   num__ing    0.041460
16            num__delta_inde    0.041426
9                    num__mat    0.040699
12                   num__ipv    0.040384
8                    num__ida    0.037786
10                   num__por    0.034250
5                    num__ieg    0.033318


In [ ]:
# GERAR ARQUIVO COM DADOS PARA TESTE NO APP

def gerar_planilha_validacao():
    
    print("Iniciando a geração da planilha de validação...")
    
    load_dotenv(find_dotenv())
    
    # Caminhos configurado no .env
    data_path = os.getenv("DATA_PATH")
    models_dir = os.getenv("MODELS")
    processed_dir = os.getenv("PROCESSED")
    
    if not all([data_path, models_dir, processed_dir]):
        print("Erro: Verifique se as variáveis DATA_PATH, MODELS e PROCESSED estão corretas no arquivo .env.")
        return
    
    # Carregar modelo e configurações
    caminho_modelo = os.path.join(models_dir, "modelo_passos_magicos.pkl")
    caminho_config = os.path.join(models_dir, "config_passos_magicos.pkl")
    
    if not os.path.exists(caminho_modelo) or not os.path.exists(caminho_config):
        print(f"Erro: Arquivos .pkl não encontrados em {models_dir}.")
        return
        
    modelo = joblib.load(caminho_modelo)
    config = joblib.load(caminho_config)
    limiar = config['threshold']
    
    # Carregar a base de dados
    if not os.path.exists(data_path):
        print(f"Erro: Base de dados não encontrada em {data_path}")
        return
        
    df_raw = pd.read_excel(data_path)
    
    # Selecionar dados aleatorios
    df_raw = df_raw.sample(n=10)
    
    # Preparar a base para passar pelo modelo
    df_processado = preparar_base(df_raw, modo_treino=False)
    
    # Remover colunas extras para igualar às features do treino
    colunas_para_ignorar = ["risco_defasagem_atual", "ano_pede"]
    X_pred = df_processado.drop(columns=[c for c in colunas_para_ignorar if c in df_processado.columns])
    
    # Fazer Predições
    probabilidades = modelo.predict_proba(X_pred)[:, 1]
    
    # Mapeamneto para os nomes e ordem do APP
    mapa_colunas = {
        'idade': 'Idade',
        'genero': 'Gênero',
        'fase_ideal': 'Fase Ideal',
        'mat': 'Matemática (MAT)',
        'por': 'Português (POR)',
        'ing': 'Inglês (ING)',
        'iaa': 'Ind. Autoavaliação (IAA)',
        'ieg': 'Ind. Engajamento (IEG)',
        'inde_2024': 'INDE Atual',
        'ips': 'Ind. Psicossocial (IPS)',
        'ipp': 'Ind. Psicopedagógico (IPP)',
        'inde_2022': 'INDE de 2 anos atrás',
        'inde_2023': 'INDE do ano passado',
        'ida': 'Indicador de Desempenho Acad. (IDA)',
        'ipv': 'Indicador de Ponto de Virada (IPV)',
        'n_av': 'Número de Avaliações'
    }
    
    # Filtrat só as colunas que existem e as copia
    colunas_existentes = [c for c in mapa_colunas.keys() if c in df_raw.columns]
    df_validacao = df_raw[colunas_existentes].copy()
    
    # Renomear para ficar igual ao App
    df_validacao.rename(columns=mapa_colunas, inplace=True)
    
    # Adicionar colunas do Modelo
    df_validacao['Threshold do Modelo'] = round(limiar, 2)
    df_validacao['Probabilidade de Risco'] = (probabilidades * 100).round(1).astype(str) + '%'
    
    # Adicionar as mensagens de resultado
    resultados = []
    recomendacoes = []
    
    for prob in probabilidades:
        if prob >= limiar:
            resultados.append("⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO")
            recomendacoes.append("👉 Recomendação: Necessário acompanhamento pedagógico e psicossocial intensificado.")
        else:
            resultados.append("✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO)")
            recomendacoes.append("👉 Recomendação: Manter acompanhamento padrão para garantir o engajamento.")
            
    df_validacao['Resultado da Análise'] = resultados
    df_validacao['Recomendação App'] = recomendacoes
    
    # Define a ordem exata das colunas baseada na tela do app (Sem Nome e RA)
    ordem_final_tela = [
        'Idade', 'Gênero', 'Fase Ideal', # 1. Dados do Aluno
        'Matemática (MAT)', 'Português (POR)', 'Inglês (ING)', # 2. Notas Acadêmicas
        'Ind. Autoavaliação (IAA)', 'Ind. Engajamento (IEG)', 'INDE Atual', # 3. Indicadores Col 1
        'Ind. Psicossocial (IPS)', 'Ind. Psicopedagógico (IPP)', # 3. Indicadores Col 2
        'INDE de 2 anos atrás', 'INDE do ano passado', # Histórico
        'Indicador de Desempenho Acad. (IDA)', 'Indicador de Ponto de Virada (IPV)', 'Número de Avaliações',
        'Threshold do Modelo', 'Probabilidade de Risco', 'Resultado da Análise', 'Recomendação App' # Saídas
    ]
    
    # Garantir que só vai ordenar colunas que realmente existem no df
    ordem_final_tela = [c for c in ordem_final_tela if c in df_validacao.columns]
    df_validacao = df_validacao[ordem_final_tela]
    
    # Salvar resultado
    excel_saida = os.path.join(processed_dir, "validacao_app_passos_magicos.xlsx")
    df_validacao.to_excel(excel_saida, index=False)
    
    # Print dinâmico baseado no tamanho do DataFrame
    print(f"✅ Arquivo gerado com sucesso!")
    print(f"Foram selecionados {len(df_validacao)} registros aleatoriamente.")
    
    # Mostrar o resultado final na tela
    return df_validacao

# Executar função
df_previa = gerar_planilha_validacao()
display(df_previa)

Iniciando a geração da planilha de validação...
✅ Arquivo gerado com sucesso!
Foram selecionados 10 registros aleatoriamente.


,Idade,Gênero,Fase Ideal,Matemática (MAT),Português (POR),Inglês (ING),Ind. Autoavaliação (IAA),Ind. Engajamento (IEG),INDE Atual,Ind. Psicossocial (IPS),Ind. Psicopedagógico (IPP),INDE de 2 anos atrás,INDE do ano passado,Indicador de Desempenho Acad. (IDA),Indicador de Ponto de Virada (IPV),Número de Avaliações,Threshold do Modelo,Probabilidade de Risco,Resultado da Análise,Recomendação App
471,11,Feminino,Fase 2 (5° e 6° ano),5.0,9.0,NaN,10.002,8.123570,8.075414,7.510,7.50000,7.431939,7.926825,7.000000,7.747500,4.0,0.35,50.5%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
2375,12,Menina,Fase 3 (7º e 8º ano),4.0,6.0,NaN,8.300,8.800000,NaN,7.500,NaN,7.101000,NaN,5.000000,5.167000,4.0,0.35,11.0%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
770,12,Masculino,Fase 3 (7° e 8° ano),4.5,5.5,4.0,9.585,7.155556,6.942278,6.885,6.87500,7.779764,6.748833,4.666667,6.216667,3.0,0.35,54.5%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1312,1900-01-08 00:00:00,Feminino,ALFA (1° e 2° ano),9.3,9.3,NaN,10.000,9.800000,NaN,7.520,7.50000,NaN,9.290200,9.300000,9.840000,2.0,0.35,0.8%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
2261,16,Menina,Fase 6 (2º EM),7.8,5.5,8.8,9.200,7.100000,NaN,5.000,NaN,6.957000,NaN,7.400000,6.667000,4.0,0.35,86.8%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1966,14,Feminino,Fase 4 (9° ano),6.3,5.7,5.0,10.000,9.500000,NaN,7.520,8.59375,7.889000,8.436908,5.700000,8.960000,4.0,0.35,8.2%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
2506,11,Menino,Fase 2 (5º e 6º ano),7.5,4.0,NaN,10.000,9.300000,NaN,7.500,NaN,7.914000,NaN,5.800000,7.611000,3.0,0.35,16.5%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
326,10,Masculino,Fase 1 (3° e 4° ano),7.5,8.0,NaN,9.502,7.857143,8.189629,7.510,7.50000,7.484594,7.681200,7.750000,8.085000,2.0,0.35,97.8%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1291,8,Feminino,ALFA (1° e 2° ano),0.0,0.0,NaN,9.000,7.600000,NaN,7.520,6.87500,NaN,6.527700,0.000000,8.340000,2.0,0.35,5.8%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
1904,14,Masculino,Fase 4 (9° ano),6.3,6.5,6.7,7.900,9.400000,NaN,2.520,8.28125,7.629000,7.727325,6.500000,8.377500,4.0,0.35,10.5%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
